# CGNet: A Convolutional Neural Network for Vegetation Change Detection

### Imports

In [1]:
import torch, sys, os
import torchvision.transforms as T
import torch.nn as nn
import torch.optim as optim
import numpy as np
from cgnet import CGNet
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, jaccard_score, f1_score
sys.path.append(os.path.abspath(".."))
from image_preprocessing.image_preprocessing import load_image_pairs_labels

### Prepare Dataset Class

In [2]:
class VegetationChangeDataset(Dataset):
    def __init__(self, image_pairs, labels, transform=None):
        """
        image_pairs: list of tuples (img1, img2) - each img shape [H, W, 3]
        labels: list of binary vegetation change maps - each label shape [H, W]
        transform: transform both images to tensor
        """
        
        self.image_pairs = image_pairs
        self.labels = labels
        self.transform = transform or T.ToTensor()

    def __len__(self):
        return len(self.image_pairs)

    def __getitem__(self, idx):
        img1, img2 = self.image_pairs[idx]
        label = self.labels[idx]

        # Apply transforms (ToTensor converts HWC to CHW)
        x1 = self.transform(img1)
        x2 = self.transform(img2)

        # Convert label to tensor
        label = torch.from_numpy(label).float().unsqueeze(0)  # [1, H, W]

        return x1, x2, label

### Define Model, Loss, and Optimizer

In [3]:
model = CGNet(pretrained=True)
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

criterion = nn.BCEWithLogitsLoss()  # Use logits for numerical stability
optimizer = optim.Adam(model.parameters(), lr=1e-4)

c:\Users\jonaf\anaconda3\envs\CGNet\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jonaf\anaconda3\envs\CGNet\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Create Training Dataset

In [4]:
image_paths = [
    ('../../Data/Antwerpen/Antwerpen_2018/JPEG2000/OMWRGB18VL_11002.jp2', '../../Data/Antwerpen/Antwerpen_2022/JPEG2000/OMWRGB22VL_11002.jp2', 8500, 7000, 4420, 6980, 3320, 5880, 256),
    ('../../Data/Leuven/Leuven_2018/JPEG2000/OMWRGB18VL_24062.jp2', '../../Data/Leuven/Leuven_2022/JPEG2000/OMWRGB22VL_24062.jp2', 8500, 7000, 3620, 6180, 2320, 4880, 256),
    ('../../Data/Kortrijk/Kortrijk_2018/JPEG2000/OMWRGB18VL_34022.jp2', '../../Data/Kortrijk/Kortrijk_2022/JPEG2000/OMWRGB22VL_34022.jp2', 8500, 7000, 2120, 4680, 1520, 4080, 256),
    ('../../Data/Brugge/Brugge_2018/JPEG2000/OMWRGB18VL_31005.jp2', '../../Data/Brugge/Brugge_2022/JPEG2000/OMWRGB22VL_31005.jp2', 8000, 6500, 4470, 7030, 2020, 4580, 256),
    ('../../Data/Hasselt/Hasselt_2018/JPEG2000/OMWRGB18VL_71022.jp2', '../../Data/Hasselt/Hasselt_2022/JPEG2000/OMWRGB22VL_71022.jp2', 8500, 7000, 2570, 5130, 3020, 5580, 256),
    ('../../Data/Mechelen/Mechelen_2018/JPEG2000/OMWRGB18VL_12025.jp2', '../../Data/Mechelen/Mechelen_2022/JPEG2000/OMWRGB22VL_12025.jp2', 8500, 7000, 3570, 6130, 3020, 5580, 256),
               ]

image_pairs_train, labels_train = load_image_pairs_labels(image_paths, normalized=False)

# cleanup
if 255 in np.unique(labels_train):
   labels_train = np.clip(labels_train, 0, 1).astype(np.uint8)

Reading ../../Data/Antwerpen/Antwerpen_2018/JPEG2000/OMWRGB18VL_11002.jp2 into shape (3, 8500, 7000)
Reading ../../Data/Antwerpen/Antwerpen_2022/JPEG2000/OMWRGB22VL_11002.jp2 into shape (3, 8500, 7000)
Reading ../../Data/Leuven/Leuven_2018/JPEG2000/OMWRGB18VL_24062.jp2 into shape (3, 8500, 7000)
Reading ../../Data/Leuven/Leuven_2022/JPEG2000/OMWRGB22VL_24062.jp2 into shape (3, 8500, 7000)
Reading ../../Data/Kortrijk/Kortrijk_2018/JPEG2000/OMWRGB18VL_34022.jp2 into shape (3, 8500, 7000)
Reading ../../Data/Kortrijk/Kortrijk_2022/JPEG2000/OMWRGB22VL_34022.jp2 into shape (3, 8500, 7000)
Reading ../../Data/Brugge/Brugge_2018/JPEG2000/OMWRGB18VL_31005.jp2 into shape (3, 8000, 6500)
Reading ../../Data/Brugge/Brugge_2022/JPEG2000/OMWRGB22VL_31005.jp2 into shape (3, 8000, 6500)
Reading ../../Data/Hasselt/Hasselt_2018/JPEG2000/OMWRGB18VL_71022.jp2 into shape (3, 8500, 7000)
Reading ../../Data/Hasselt/Hasselt_2022/JPEG2000/OMWRGB22VL_71022.jp2 into shape (3, 8500, 7000)
Reading ../../Data/Mechele

### Creating Test Dataset

In [5]:
image_paths = [
    ('../../Data/Gent/Gent_2020/JPEG2000/OMWRGB20VL_44021.jp2', '../../Data/Gent/Gent_2024/JPEG2000/OMWRGB24VL_44021.jp2', 8500, 7000, 4220, 6780, 2520, 5080, 256)
               ]

test_image_pairs, test_labels = load_image_pairs_labels(image_paths, normalized=False)

# cleanup
if 255 in np.unique(test_labels):
   test_labels = np.clip(test_labels, 0, 1).astype(np.uint8)

Reading ../../Data/Gent/Gent_2020/JPEG2000/OMWRGB20VL_44021.jp2 into shape (3, 8500, 7000)


Reading ../../Data/Gent/Gent_2024/JPEG2000/OMWRGB24VL_44021.jp2 into shape (3, 8500, 7000)


### Fine-Tuning Loop (Training)

In [6]:
# Prepare data
train_dataset = VegetationChangeDataset(image_pairs_train, labels_train)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

for epoch in range(10):  # You can increase this
    model.train()
    running_loss = 0.0

    for x1, x2, label in train_loader:
        x1 = x1.to(device)
        x2 = x2.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        change_map, _ = model(x1, x2)  # Use only the first output
        loss = criterion(change_map, label)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")

Epoch 1, Loss: 0.5220
Epoch 2, Loss: 0.4510
Epoch 3, Loss: 0.4250
Epoch 4, Loss: 0.4021
Epoch 5, Loss: 0.3738
Epoch 6, Loss: 0.3547
Epoch 7, Loss: 0.3372
Epoch 8, Loss: 0.3322
Epoch 9, Loss: 0.3157
Epoch 10, Loss: 0.3050


### Evaluate The Model

In [7]:
def evaluate_model(model, dataloader, threshold=0.5, device='cpu'):
    model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for x1, x2, labels in dataloader:
            x1 = x1.to(device)
            x2 = x2.to(device)
            labels = labels.to(device)

            preds, _ = model(x1, x2)  # [B, 1, H, W]
            preds = torch.sigmoid(preds).squeeze(1)  # [B, H, W]
            labels = labels.squeeze(1)

            # Binarize predictions
            preds_bin = (preds > threshold).int().cpu().numpy()
            labels_bin = labels.int().cpu().numpy()

            # Flatten and collect
            for p, l in zip(preds_bin, labels_bin):
                all_preds.append(p.flatten())
                all_labels.append(l.flatten())

    # Concatenate all predictions
    y_pred = np.concatenate(all_preds)
    y_true = np.concatenate(all_labels)

    # Compute metrics
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    iou = jaccard_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    return {
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "IoU": iou,
        "f1": f1,
    }, y_pred, y_true

In [9]:
# Prepare test set
test_dataset = VegetationChangeDataset(test_image_pairs, test_labels)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Load model to device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# Evaluate
metrics, y_pred, y_true = evaluate_model(model, test_loader, device=device)

# Show results
for name, value in metrics.items():
    print(f"{name}: {value:.4f}")

Accuracy: 0.7736
Precision: 0.5432
Recall: 0.1547
IoU: 0.1369
f1: 0.2408


### Save the Fine-Tuned Model

In [13]:
torch.save(model.state_dict(), "cgnet_vegetation_finetuned.pth")